# 좌표로 행정동 가져오기

## 버스 정류장

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
bus_stop = pd.read_csv('서울시버스정류소좌표데이터(2022.08.24).csv', encoding='CP949')
bus_stop.head()

,NODE_ID,ARS-ID,정류소명,좌표X,좌표Y
0,100000001,1001,종로2가사거리,126.987786,37.569764
1,100000002,1002,창경궁.서울대학교병원,126.996520,37.579179
2,100000003,1003,명륜3가.성대입구,126.998290,37.582709
3,100000004,1004,종로2가.삼일교,126.987507,37.568582
4,100000005,1005,혜화동로터리.여운형활동터,127.001694,37.586230


In [3]:
def juso(x) :
    method = "GET"
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
    params = {'x' : x['좌표X'], 'y' : x['좌표Y']}
    header = {'authorization': 'KakaoAK apikey', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}
    
    try :
        response = requests.request(method=method, url=url, headers=header, params=params )
        tokens = response.json()
        x['시도'] = tokens['documents'][1]['region_1depth_name']
        x['시군구'] = tokens['documents'][1]['region_2depth_name']
        x['동'] = tokens['documents'][1]['region_3depth_name']
    except :
        x['시도'] = ''
        x['시군구'] = ''
        x['동'] = ''
    return x

In [4]:
bus_stop = bus_stop.apply(juso, axis =1)

In [5]:
bus_stop.head()

,NODE_ID,ARS-ID,정류소명,좌표X,좌표Y,시도,시군구,동
0,100000001,1001,종로2가사거리,126.987786,37.569764,서울특별시,종로구,종로1.2.3.4가동
1,100000002,1002,창경궁.서울대학교병원,126.996520,37.579179,서울특별시,종로구,종로1.2.3.4가동
2,100000003,1003,명륜3가.성대입구,126.998290,37.582709,서울특별시,종로구,혜화동
3,100000004,1004,종로2가.삼일교,126.987507,37.568582,서울특별시,종로구,종로1.2.3.4가동
4,100000005,1005,혜화동로터리.여운형활동터,127.001694,37.586230,서울특별시,종로구,혜화동


In [6]:
bus_stop.to_csv('버스 정류장 주소.csv', index=False, encoding='CP949')

## 지하철역

In [7]:
subway = pd.read_csv('station_coordinate.csv', names = ['호선', '역 이름', 'ID', '좌표Y', '좌표X'], skiprows=1)
subway.head()

,호선,역 이름,ID,좌표Y,좌표X
0,01호선,녹양,1908.0,37.759380,127.042292
1,01호선,남영,1002.0,37.541021,126.971300
2,01호선,용산,1003.0,37.529849,126.964561
3,01호선,노량진,1004.0,37.514219,126.942454
4,01호선,대방,1005.0,37.513342,126.926382


In [8]:
subway = subway.apply(juso, axis =1)

In [9]:
subway.head()

,호선,역 이름,ID,좌표Y,좌표X,시도,시군구,동
0,01호선,녹양,1908.0,37.759380,127.042292,경기도,의정부시,녹양동
1,01호선,남영,1002.0,37.541021,126.971300,서울특별시,용산구,남영동
2,01호선,용산,1003.0,37.529849,126.964561,서울특별시,용산구,한강로동
3,01호선,노량진,1004.0,37.514219,126.942454,서울특별시,동작구,노량진1동
4,01호선,대방,1005.0,37.513342,126.926382,서울특별시,영등포구,신길7동


In [11]:
subway.to_csv('지하철역 주소.csv', index=False, encoding='CP949')